# Grad-CAM Explainability

This notebook demonstrates how to generate Grad-CAM visualizations:
- Loading a trained model
- Generating attention heatmaps
- Visualizing model focus regions

In [1]:
# Import libraries
import os
import sys
import torch
import torch.nn.functional as F
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# Add parent directory to path
sys.path.insert(0, os.path.abspath('..'))

from src.model import LungCancerClassifier
from src.gradcam import visualize_gradcam, GradCAM
from src.dataset import LungCancerDataset

print("Libraries imported successfully!")

Libraries imported successfully!


## Load Trained Model

In [2]:
# TODO: Update model path
MODEL_PATH = "../results/best_model.pth"
BACKBONE = "resnet18"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize model
model = LungCancerClassifier(
    num_classes=3,
    backbone=BACKBONE,
    pretrained=False
)

# Load checkpoint
if os.path.exists(MODEL_PATH):
    checkpoint = torch.load(MODEL_PATH, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    print(f"Model loaded from {MODEL_PATH}")
    print(f"Validation accuracy: {checkpoint.get('val_acc', 'N/A'):.2f}%")
else:
    print(f"Model checkpoint not found at {MODEL_PATH}")
    print("Using untrained model for demonstration")

model.to(device)
model.eval()

/Users/jaiyandh/Projects/lungcancer/.venv/lib/python3.14/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/jaiyandh/Projects/lungcancer/.venv/lib/python3.14/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Model checkpoint not found at ../results/best_model.pth
Using untrained model for demonstration


LungCancerClassifier(
  (feature_extractor): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, tr

## Generate Grad-CAM Visualizations

In [3]:
# Load sample images
DATA_DIR = "../data/raw"  # TODO: Update this path
class_names = ["Normal", "Benign", "Malignant"]

# Get sample image from dataset
try:
    dataset = LungCancerDataset(DATA_DIR, split="val")
    
    # Get one sample per class
    samples_per_class = {}
    for i in range(len(dataset)):
        _, label = dataset[i]
        if label not in samples_per_class:
            # Get original image (before normalization)
            # Note: In a real scenario, you'd want to store original images
            samples_per_class[label] = i
            if len(samples_per_class) == 3:
                break
    
    print(f"Found {len(samples_per_class)} samples for Grad-CAM visualization")
    
except Exception as e:
    print(f"Error loading dataset: {e}")
    print("Cannot generate Grad-CAM visualizations without dataset")
    samples_per_class = {}

Found 3 samples for Grad-CAM visualization


In [4]:
# Generate Grad-CAM for each class
if samples_per_class:
    for label, idx in samples_per_class.items():
        # Get preprocessed image
        image_tensor, _ = dataset[idx]
        
        # TODO: Get original image array (you may need to load separately)
        # For now, using a placeholder
        from torchvision import transforms
        denormalize = transforms.Normalize(
            mean=[-0.485/0.229, -0.456/0.224, -0.406/0.225],
            std=[1/0.229, 1/0.224, 1/0.225]
        )
        original_image = denormalize(image_tensor).clamp(0, 1)
        original_image_array = original_image.permute(1, 2, 0).cpu().numpy()
        
        # Generate Grad-CAM
        save_path = f"../results/gradcam_samples/gradcam_{class_names[label]}.png"
        visualize_gradcam(
            model,
            image_tensor,
            original_image_array,
            class_names,
            save_path=save_path,
            device=device
        )
        
        print(f"Grad-CAM visualization saved for {class_names[label]}")
else:
    print("No samples available for Grad-CAM visualization")

Grad-CAM visualization saved to ../results/gradcam_samples/gradcam_Normal.png
Grad-CAM visualization saved for Normal
Grad-CAM visualization saved to ../results/gradcam_samples/gradcam_Benign.png
Grad-CAM visualization saved for Benign
Grad-CAM visualization saved to ../results/gradcam_samples/gradcam_Malignant.png
Grad-CAM visualization saved for Malignant
